In [5]:
import torch
import torch.nn as nn
from zytlib import vector
import math
from zytlib.classfunc import save_args
from torch.nn.utils.rnn import pad_sequence
from zytlib.wrapper import second_argument
from zytlib.table import table
import math
from torchfunction.utils import seed_torch
import random

In [6]:
from dataset import Emulater, emulate, generate_in_train_label, dataset, SimulatedDataset

In [7]:
delta_t = 20
mean_delay = 300
mean_rank = 200
mean_last_delay = 1000
num_items = 2
random_seed = 1024

info = table(delta_t=delta_t, mean_delay=mean_delay, mean_rank=mean_rank, mean_last_delay=mean_last_delay, num_items=num_items, random_seed=random_seed)

# generate training dataset

In [12]:
seed_torch(random_seed)
rank3_items_list = vector.range(num_items) ** 3
rank2_items_list = vector.range(num_items) ** 2
rank1_items_list = vector.range(num_items) ** 1

def map_delay(delay):
    return delay[:-1].map(lambda x: int(x * (random.random() + 0.5))) + delay[-1:].map(lambda x: int(x * (random.random() * 0.5 + 1)))

In [13]:
map_delay(vector(100,100,100))

[129, 98, 119]

In [14]:
rank1_items_list

[(0,), (1,)]

In [15]:
rank2_items_list

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [16]:
rank3_items_list

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]

In [18]:
rank3_dataset = vector()
for items in rank3_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank3_dataset.append((items, delay, rank))

rank2_dataset = vector()
for items in rank2_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank2_dataset.append((items, delay, rank))

rank1_dataset = vector()
for items in rank1_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank1_dataset.append((items, delay, rank))

In [22]:
final_rank2_dataset_train = vector()
final_rank2_dataset_test = dict()
final_rank2_dataset_test["rank1"] = vector()
final_rank2_dataset_test["rank2"] = vector()

# for items, delay, rank in rank2_dataset:
#     final_rank2_dataset_train.append((items, delay.map(lambda x: int(x * (random.random() + 0.5))), rank.map(lambda x: int(x * (random.random() + 0.5)))))
#     final_rank2_dataset_test["rank{}".format(len(items))].append((items, delay.map(lambda x: int(x * (random.random() + 0.5))), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     

for items, delay, rank in rank1_dataset:
    final_rank2_dataset_test["rank{}".format(len(items))].append((items, delay, rank, 1))     

for items, delay, rank in rank2_dataset:
    final_rank2_dataset_test["rank{}".format(len(items))].append((items, delay, rank, 1))     
 
table(info=info, train=vector(), test=final_rank2_dataset_test).save(f"dataset/dataset_item_{num_items}_test_rank12.db")

In [23]:
final_rank3_dataset_train = vector()
final_rank3_dataset_test = dict()
final_rank3_dataset_test["rank1"] = vector()
final_rank3_dataset_test["rank2"] = vector()
final_rank3_dataset_test["rank3"] = vector()

# for items, delay, rank in rank2_dataset:
#     final_rank2_dataset_train.append((items, delay.map(lambda x: int(x * (random.random() + 0.5))), rank.map(lambda x: int(x * (random.random() + 0.5)))))
#     final_rank2_dataset_test["rank{}".format(len(items))].append((items, delay.map(lambda x: int(x * (random.random() + 0.5))), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     

for items, delay, rank in rank1_dataset:
    final_rank3_dataset_test["rank{}".format(len(items))].append((items, delay, rank, 1))     

for items, delay, rank in rank2_dataset:
    final_rank3_dataset_test["rank{}".format(len(items))].append((items, delay, rank, 1))     
    
for items, delay, rank in rank3_dataset:
    final_rank3_dataset_test["rank{}".format(len(items))].append((items, delay, rank, 1))     
 
table(info=info, train=vector(), test=final_rank3_dataset_test).save(f"dataset/dataset_item_{num_items}_test_rank123.db")

In [ ]:
final_rank2_dataset_test

In [62]:
rank2_dataset = vector()
for items in rank2_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank2_dataset.append((items, delay, rank))

rank1_dataset = vector()
for items in rank1_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank1_dataset.append((items, delay, rank))

In [63]:
final_rank2_dataset_train = vector()
final_rank2_dataset_test = dict()
final_rank2_dataset_test["rank1"] = vector()
final_rank2_dataset_test["rank2"] = vector()

for items, delay, rank in rank2_dataset:
    for _ in range(70 * 3):
        final_rank2_dataset_train.append((items, delay.map(lambda x: int(x * (random.random() + 0.5))), rank.map(lambda x: int(x * (random.random() + 0.5)))))
    for _ in range(30 * 3):
        final_rank2_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     

for items, delay, rank in rank1_dataset:
    for _ in range(30 * 3):
        final_rank2_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     
 
table(info=info, train=final_rank2_dataset_train, test=final_rank2_dataset_test).save(f"dataset/dataset_item_{num_items}_train_rank2.db")

In [64]:
final_rank2_dataset_train.length
final_rank2_dataset_train.sample(10)

[((0, 1), [9, 17, 71], [12, 12]), ((0, 1), [16, 19, 64], [13, 7]), ((1, 0), [21, 8, 68], [5, 6]), ((1, 0), [20, 17, 64], [6, 13]), ((0, 1), [21, 19, 28], [14, 8]), ((1, 1), [11, 15, 60], [14, 8]), ((0, 0), [15, 11, 40], [8, 6]), ((1, 0), [16, 10, 58], [6, 9]), ((0, 1), [10, 20, 38], [10, 12]), ((1, 1), [11, 17, 27], [11, 5])], with index mapping

In [5]:
rank2_dataset = vector()
for items in rank2_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank2_dataset.append((items, delay, rank))

rank1_dataset = vector()
for items in rank1_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank1_dataset.append((items, delay, rank))
    
rank3_dataset = vector()
for items in rank3_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank3_dataset.append((items, delay, rank))

In [48]:
final_rank3_dataset_train = vector()
final_rank3_dataset_test = dict()
final_rank3_dataset_test["rank1"] = vector()
final_rank3_dataset_test["rank2"] = vector()
final_rank3_dataset_test["rank3"] = vector()
seed_torch(random_seed)
length = 3

for items, delay, rank in rank3_dataset:
    for _ in range(70 * 3):
        final_rank3_dataset_train.append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5)))))
    for _ in range(30 * 3):
        final_rank3_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     

for items, delay, rank in rank2_dataset:
    for _ in range(30 * 3):
        final_rank3_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     

for items, delay, rank in rank1_dataset:
    for _ in range(30 * 3):
        final_rank3_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     
 
table(info=info, train=final_rank3_dataset_train, test=final_rank3_dataset_test).save(f"dataset/dataset_item_{num_items}_train_length_{length}.db")

In [67]:
final_rank3_dataset_train = vector()
final_rank3_dataset_test = dict()
final_rank3_dataset_test["rank1"] = vector()
final_rank3_dataset_test["rank2"] = vector()
final_rank3_dataset_test["rank3"] = vector()
seed_torch(random_seed)
length = 3

for items, delay, rank in rank3_dataset + rank2_dataset + rank1_dataset:
    for _ in range(70 * 3):
        final_rank3_dataset_train.append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5)))))
    for _ in range(30 * 3):
        final_rank3_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     

# for items, delay, rank in rank2_dataset:
#     for _ in range(30 * 3):
#         final_rank3_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     

# for items, delay, rank in rank1_dataset:
#     for _ in range(30 * 3):
#         final_rank3_dataset_test["rank{}".format(len(items))].append((items, map_delay(delay), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))     
 
table(info=info, train=final_rank3_dataset_train, test=final_rank3_dataset_test).save(f"dataset/dataset_item_{num_items}_train_length_123.db")

In [68]:
sd = SimulatedDataset(f"dataset/dataset_item_{num_items}_train_length_123.db", 2, 10, train_items_crop=2)

In [72]:
sd

Dataset: 
	train:	2940	[((0, 1, 0), [11, 11, 13, 57], [9, 14, 6])]
	test:
	rank1	180	[((1,), [16, 56], [9], 1)]
	rank2	360	[((1, 0), [21, 20, 66], [6, 5], 1)]
	rank3	720	[((1, 0, 1), [21, 17, 18, 51], [14, 12, 12], 1)]

In [42]:
seed_torch(random_seed)
rank12_dataset = vector()
for items in rank1_items_list:
    # for _ in range(num_items):
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank12_dataset.append((items, delay, rank))
    
for items in rank2_items_list:
    rank = vector([math.ceil(mean_rank / delta_t)] * len(items))
    delay = vector([math.ceil(mean_delay / delta_t)] * len(items)) + [math.ceil(mean_last_delay / delta_t)]
    rank12_dataset.append((items, delay, rank))

In [ ]:
vector.range(10).map(lambda x: x * (random.random() + 0.5))

In [ ]:
final_rank12_dataset_train = vector()
final_rank12_dataset_test = dict()
final_rank12_dataset_test["rank1"] = vector()
final_rank12_dataset_test["rank2"] = vector()

for items, delay, rank in rank12_dataset:
    for _ in range(70 * 3):
        final_rank12_dataset_train.append((items, delay.map(lambda x: int(x * (random.random() + 0.5))), rank.map(lambda x: int(x * (random.random() + 0.5)))))
    for _ in range(30 * 3):
        final_rank12_dataset_test["rank{}".format(len(items))].append((items, delay.map(lambda x: int(x * (random.random() + 0.5))), rank.map(lambda x: int(x * (random.random() + 0.5))), 1))       
table(info=info, train=final_rank12_dataset_train, test=final_rank12_dataset_test).save(f"dataset/dataset_item_{num_items}_rank12.db")

In [ ]:
print(len(final_rank12_dataset_train))
print(final_rank12_dataset_train.map(lambda x: x[0]).count_all())

# generate testing dataset

In [ ]:
items_list = vector.range(num_items) ** 2
rank2_dataset = vector()
for items in items_list:
    rank = vector([10]) * 2
    delay = vector([15]) * 2
    for d in range(25, 101):
        rank2_dataset.append((items, delay + [d], rank, d))

In [ ]:
rank2_dataset.sample(5).apply(print)

In [ ]:
rank2_dataset.length

In [ ]:
t = table(train=vector(), test=rank2_dataset)
t.save(f"dataset/dataset_only_rank2_with_lastdelay_25_100.db")

In [ ]:
items_list = vector.range(num_items) ** 2
rank2_dataset = vector()
for items in items_list:
    rank = vector([10]) * 2
    delay = vector([15]) * 2
    d = 50
    rank2_dataset.append((items, delay + [d], rank, d))

In [ ]:
rank2_dataset.sample(5).apply(print)

In [ ]:
rank2_dataset.length

In [ ]:
t.save(f"dataset/dataset_only_rank2_with_lastdelay_50.db")

In [ ]:
items_list = vector.range(num_items) ** 1
rank1_dataset = vector()
for items in items_list:
    rank = vector([10])
    delay = vector([15])
    for d in range(25, 101):
        rank1_dataset.append((items, delay + [d], rank, d))

In [ ]:
len(rank1_dataset), rank1_dataset.sample()

In [ ]:
t1 = table(train=vector(), test=rank1_dataset)
t1.save(f"dataset/dataset_only_rank1_with_lastdelay_25_100.db")

In [ ]:
items_list = vector.range(num_items) ** 1
rank1_dataset_new= vector()
for items in items_list:
    rank = vector([10])
    delay = vector([15])
    for r in range(5, 16):
        rank1_dataset_new.append((items, vector([15, 50]), vector([r]), r))

In [ ]:
len(rank1_dataset_new), rank1_dataset_new.sample()

In [ ]:
t2 = table(train=vector(), test=rank1_dataset_new)
t2.save(f"dataset/dataset_only_rank1_with_last_delay_50_rank_5_15.db")

In [ ]:
items_list = vector.range(num_items) ** 3
# rank3_dataset = vector()
delay = int(mean_delay / delta_t)
rank = int(mean_rank / delta_t)
last_delay = int(mean_last_delay / delta_t) * 2
    
rank3_dataset = items_list.map(lambda items: (items, vector([delay, delay, delay, last_delay]), vector([rank, rank, rank]), 0))

In [ ]:
len(rank3_dataset), rank3_dataset.sample()

In [ ]:
t3 = table(train=vector(), test=rank3_dataset)
t3.save("dataset/dataset_only_rank3_with_rank_10_delay_15_last_delay_100.db")

In [ ]:
items_list = vector.range(num_items) ** 1

delay = int(mean_delay / delta_t)
rank = int(mean_rank / delta_t)
last_delay = int(mean_last_delay / delta_t) * 3

rank4_dataset = items_list.map(lambda items: (items, vector([delay, last_delay]), vector([rank]), 0))

In [ ]:
len(rank4_dataset), rank4_dataset.sample()

In [ ]:
t4 = table(train=vector(), test=rank4_dataset)
t4.save("dataset/dataset_only_rank1_with_rank_10_delay_15_last_delay_150.db")

In [ ]:
items_list = vector.range(num_items) ** 4
# rank3_dataset = vector()
delay = int(mean_delay / delta_t)
rank = int(mean_rank / delta_t)
last_delay = int(mean_last_delay / delta_t) * 2
    
length4_dataset = items_list.map(lambda items: (items, vector([delay, delay, delay, delay, last_delay]), vector([rank, rank, rank, rank]), 0))

In [ ]:
len(length4_dataset), length4_dataset.sample()

In [ ]:
t5 = table(train=vector(), test=length4_dataset)
t5.save("dataset/dataset_only_rank4_with_rank_10_delay_15_last_delay_100.db")

In [ ]:
items_list = vector.range(num_items) ** 3
# rank3_dataset = vector()
delay = int(mean_delay / delta_t)
rank = int(mean_rank / delta_t)
last_delay = int(mean_last_delay / delta_t)
    
rank3_dataset = items_list.map(lambda items: (items, vector([delay, delay, delay, last_delay]), vector([rank, rank, rank]), 0))

print(len(rank3_dataset), rank3_dataset.sample())

t6 = table(train=vector(), test=rank3_dataset)
t6.save("dataset/dataset_only_rank3_with_rank_10_delay_15_last_delay_50.db")

In [65]:
hyper=table(datapath='dataset/dataset_item_2_train_length_3.db', learning_rate=0.001, l2_reg=1e-5, encoder_max_rank=2, max_epochs=3000, embedding='dataset/embedding_inputdim_2_embeddingdim_4096_round_without_normalize.db', residual_loss=1e-2, train_items_crop=2)

In [66]:
hyper

{'datapath': 'dataset/dataset_item_2_train_length_3.db',
 'learning_rate': 0.001,
 'l2_reg': 1e-05,
 'encoder_max_rank': 2,
 'max_epochs': 3000,
 'embedding': 'dataset/embedding_inputdim_2_embeddingdim_4096_round_without_normalize.db',
 'residual_loss': 0.01,
 'train_items_crop': 2}